# Puffer

## Bibliotheken importieren

In [1]:
import warnings
warnings.filterwarnings('ignore')

from shapely.geometry import *

import osmnx as ox
import networkx as nx
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

Den zu analysierenden Netzwerkgraphen einladen und ins entsprechende Koordiantensystem projizieren.

In [2]:
G = ox.graph_from_xml('./Daten/osm/gesamtgraph.osm', simplify=False)
G = ox.project_graph(G, to_crs='EPSG:25832')

Um später Puffer um entsprechende Bereiche des Graphens zu legen, wird eine Spalte 'time' hinzugefügt, die jeder Kante einen enstprechenden Wiederstand definiert. Dieser entspricht der Reisezeit, die für die Kante unter Berücksichtigung der Geschwindigkeit gebraucht wird.

In [3]:
# Iterieren durch jede Kante im Graphen
for u, v, data in G.edges(data=True):
    # Konvertierung 'maxspeed' von String zu Float
    maxspeed = float(data['maxspeed'])

    # Berechnen der Zeit in Minuten und hinzufügen als 'time' zum Kantenattribut
    data['time'] = (data['length'] / maxspeed) * (60 / 1000)

## Ärzte einladen

Einladen der Daten und Geometrien für die Ärzte. Die Koordianten werden wieder ins EPSG 25832 transformiert.

In [7]:
aerzte = gpd.read_file('./Daten/Aerzte/aerzte_general_sh.geojson')

## Buffer erstellen

### Buffer Funktion

Funktion zur Puffer-Erstellung

Mit der Funktion buffer können Puffer um das Netzwerk geladen werden. Diese werden entsprechend der vorher angegebenen Fahrzeit von den Ausgangspunkten aus erstellt und nur um das innerhalb der Zeit zu erreichende Netzwerk berechnet. Die Größe der Puffer kann variabel gewählt werden. 


Eingabe: 
- startpunkte : GeoDataFrame
- zeit : integer
- buffer : integer
- speichername : string

Ausgabe: 
- gdf : geoDataFrame

In [18]:
def buffer(startpunkte,zeit,buffer, speichername):
    gdf = gpd.GeoDataFrame()
    # Iterieren über alle Einträge im GDF "aerzte"
    for i in startpunkte.geometry:
        start = Point(i.x, i.y)
        s_p = ox.nearest_nodes(G, start.x, start.y)

        subgraph = nx.ego_graph(G, s_p, radius=zeit, distance='time')
        subgraph_edges = ox.graph_to_gdfs(subgraph, nodes=False, edges=True)

        # Erstellen eines Puffer um die Kanten
        buffer_radius = buffer  # in Metern
        subgraph_edges['geometry'] = subgraph_edges['geometry'].buffer(buffer_radius)
        gdf_edges = subgraph_edges[['geometry']]
        
        # Zusammenführen der Geometrien aus gdf_edges zu einer einzelnen Geometrie 
        merged_geometry = gdf_edges.unary_union

        # Erstellen eins neues Geodataframes mit der zusammengeführten Geometrie
        gdf_merged = gpd.GeoDataFrame(geometry=[merged_geometry])

        gdf = pd.concat([gdf, gdf_merged], ignore_index=True)

    gdf.to_file('./Puffer/'+speichername+'.geojson', driver = 'GeoJSON', crs='EPSG:25832')
    return gdf

In [23]:
gdf5 = buffer(aerzte,5,200,'buffer5')
gdf10 = buffer(aerzte,10,400,'buffer10')
gdf15 = buffer(aerzte,15,600,'buffer15')
gdf20 = buffer(aerzte,20,800,'buffer20')

## Kreise und Einwohner einlesen

In [11]:
kreise_gdf = gpd.read_file("./Daten/Bevoelkerung/AnteilBevoelkerung2021_ab64_4326.geojson")

## Visualisiern 

In [ ]:
fig, ax = plt.subplots(figsize=(20, 20))
kreise_gdf.plot(ax=ax, color= 'darkgrey', alpha=0.8)
gdf20.plot(ax=ax, color='red', alpha=0.9)
gdf15.plot(ax=ax, color='orange', alpha=0.9)
gdf10.plot(ax=ax, color='yellow', alpha=0.9)
gdf5.plot(ax=ax, color='green', alpha=0.9)
#aerzte.plot(ax=ax, color='black', markersize = 10)
#ox.plot_graph(G, ax=ax, node_color='red', node_size=0, edge_linewidth=0.5, edge_color='#999999')

ax.set_axis_off()
# Zeige den Plot an
plt.show()

: 